In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import glob

In [2]:
##########################################
# source:
# https://github.com/theRealSuperMario/supermariopy/blob/master/scripts/tflogs2pandas.py
#########################################
import tensorflow as tf
import glob
import os
import pandas as pd
import traceback
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import click
import pprint


# Extraction function
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    
    Parameters
    ----------
    path : str
        path to tensorflow log file
    
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame()
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            if tag == "rollout/return" or tag == "rollout/Q_mean": #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Made change here
                event_list = event_acc.Scalars(tag)
                values = list(map(lambda x: x.value, event_list))
                step = list(map(lambda x: x.step, event_list))
                r = pd.Series(data=values, index=step, name=tag)
                runlog_data = pd.concat([runlog_data, r], axis=1)
    # Dirty catch of DataLossError
    except:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data


def many_logs2pandas(event_paths):
    all_logs = pd.DataFrame()
    for path in event_paths:
        log = tflog2pandas(path)
        if log is not None:
            if all_logs.shape[0] == 0:
                all_logs = log
            else:
                all_logs = all_logs.append(log, ignore_index=True)
    return all_logs


def extract_tf_to_csv(logfile: str, write_pkl: bool, write_csv: bool, out_dir: str):
    """This is a enhanced version of https://gist.github.com/ptschandl/ef67bbaa93ec67aba2cab0a7af47700b
    This script exctracts variables from all logs from tensorflow event files ("event*"),
    writes them to Pandas and finally stores them a csv-file or pickle-file including all (readable) runs of the logging directory.
    Example usage:
    # create csv file from all tensorflow logs in provided directory (.) and write it to folder "./converted"
    tflogs2pandas.py . --csv --no-pkl --o converted
    # creaste csv file from tensorflow logfile only and write into and write it to folder "./converted"
    tflogs2pandas.py tflog.hostname.12345 --csv --no-pkl --o converted
    """
    pp = pprint.PrettyPrinter(indent=4)
    if os.path.isfile(logfile):
        event_paths = [logfile]
    else:
        raise ValueError(
            "input argument {} has to be a file".format(
                logfile
            )
        )
    # Call & append
    if event_paths:
        pp.pprint("Found tensorflow logs to process:")
        pp.pprint(event_paths)
        all_logs = many_logs2pandas(event_paths)
        pp.pprint("Head of created dataframe")
        pp.pprint(all_logs.head())

        os.makedirs(out_dir, exist_ok=True)
        run_id = logfile.split(".")[5]+logfile.split(".")[6]
        mode = logfile.split("/")[-1].split("\\")[1]
        datetime = logfile.split("/")[-1].split("\\")[2]
        if write_csv:
            print("saving to csv file")
            out_file = os.path.join(out_dir, "run_"+mode+"_"+datetime+"_"+run_id+"_file.csv")
            print(out_file)
            all_logs.to_csv(out_file, index=None)
        if write_pkl:
            print("saving to pickle file")
            out_file = os.path.join(out_dir, "all_training_logs_in_one_file.pkl")
            print(out_file)
            all_logs.to_pickle(out_file)
    else:
        print("No event paths have been found.")

In [31]:
# folders = ["point125","point25","point5","point75", "one"]
# folders = ["one"]
folders = ["vzero","vpoint01","vpoint05","vpoint1","vpoint3"]
# folders = ["ddpg"]
event_files = dict()
for folder in folders:
    
    event_files[folder] = glob.glob("../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/"+folder+"/**/events*", recursive=True)
    event_files[folder].sort()
    print(folder+": {} files:".format(len(event_files[folder])))
    for f in event_files[folder]:
            print(f)

vzero: 13 files:
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-10-22-104812-0-8\summary\tb\events.out.tfevents.1571755716.gra1057
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-10-22-104815-1-8\summary\tb\events.out.tfevents.1571755722.gra1063
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-10-23-161114-1-8\summary\tb\events.out.tfevents.1571861498.gra701
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-10-23-161120-0-8\summary\tb\events.out.tfevents.1571861505.gra641
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-11-16-160331-0-50\summary\tb\events.out.tfevents.1573938236.gra105
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-11-16-160331-1-50\summary\tb\events.out.tfevents.1573938235.gra125
../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\PLA\2019-11-16-160333-2-50\summary\tb\events.out.tfevents.1573938237.gra1141
../SHARCNET/Results/multi

In [33]:
out_root_dir = "../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/"
for folder, files in event_files.items():
    out_dir = out_root_dir+folder
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    for f in files:
        extract_tf_to_csv(logfile=f, write_pkl=False, write_csv=True, out_dir=out_dir)
        

'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\\PLA\\2019-10-22-104812-0-8\\summary\\tb\\events.out.tfevents.1571755716.gra1057']
'Head of created dataframe'
   rollout/Q_mean  rollout/return
1       -0.003682       20.683235
2        0.364317       31.133663
3        0.648380       37.990845
4        0.856054       31.796728
5        0.964011       38.509167
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vzero\run_PLA_2019-10-22-104812-0-8_1571755716gra1057_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vzero\\PLA\\2019-10-22-104815-1-8\\summary\\tb\\events.out.tfevents.1571755722.gra1063']
'Head of created dataframe'
   rollout/Q_mean  rollout/return
1       -0.004964       25.048233
2        0.371619       42.591736
3        0.716366       42.209415
4        1.015300       29.557184
5        1.141157       26.661417
saving to 

'Head of created dataframe'
   rollout/Q_mean  rollout/return
1        0.005179       24.053308
2        0.075120       23.648243
3        0.144604       37.751064
4        0.216974       39.395153
5        0.293078       33.045410
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint01\run_PLA_2019-11-17-182132-2-71_1574032916gra37_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vpoint01\\Random\\2019-11-16-173533-1-30\\summary\\tb\\events.out.tfevents.1573943754.gra1096']
'Head of created dataframe'
   rollout/return
1       23.758877
2       25.393217
3       25.437983
4       25.216862
5       35.625053
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint01\run_Random_2019-11-16-173533-1-30_1573943754gra1096_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vpoint01\\Random\\2019-

'Head of created dataframe'
   rollout/Q_mean  rollout/return
1        0.006754       20.540066
2        0.067527       28.665434
3        0.130916       32.030968
4        0.197648       31.172409
5        0.273268       25.776773
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint05\run_SARA_2019-11-18-174634-2-90_1574117221gra583_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vpoint1\\PLA\\2019-10-22-104957-0-0\\summary\\tb\\events.out.tfevents.1571755821.gra646']
'Head of created dataframe'
   rollout/Q_mean  rollout/return
1        0.018798       16.414875
2        0.305263       20.304504
3        0.457710       20.869877
4        0.556658       21.893898
5        0.643957       19.840298
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint1\run_PLA_2019-10-22-104957-0-0_1571755821gra646_file.csv
'Found tensorflow logs to process

'Head of created dataframe'
   rollout/return
1       10.783479
2       13.622685
3       14.225783
4       12.755690
5       13.090559
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint3\run_Random_2019-11-17-184531-1-100_1574034352gra287_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vpoint3\\Random\\2019-11-17-184531-2-100\\summary\\tb\\events.out.tfevents.1574034352.gra1115']
'Head of created dataframe'
   rollout/return
1       11.256672
2       13.927820
3       13.606174
4       14.273885
5       17.378885
saving to csv file
../SHARCNET/Results/Tensorboard_to_CSV/multi/lazy_visitor_w_randomess/old/vpoint3\run_Random_2019-11-17-184531-2-100_1574034352gra1115_file.csv
'Found tensorflow logs to process:'
[   '../SHARCNET/Results/multi/lazy_visitor_w_randomess/old/vpoint3\\SARA\\2019-10-22-104910-0-0\\summary\\tb\\events.out.tfevents.1571755774.gra649']
'Head of created datafra